<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: <https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view> 

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [32]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score
from tensorflow import keras
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

In [4]:
pd.options.display.max_columns=50

df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
df.drop(columns='customerID', inplace=True)
print(df.shape)
df.head()

(7043, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [7]:
df.describe()

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,PaperlessBilling,MonthlyCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,0.483033,0.299588,32.371149,0.903166,0.592219,64.761692,0.265370
std,0.368612,0.499748,0.458110,24.559481,0.295752,0.491457,30.090047,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000
25%,0.000000,0.000000,0.000000,9.000000,1.000000,0.000000,35.500000,0.000000
50%,0.000000,0.000000,0.000000,29.000000,1.000000,1.000000,70.350000,0.000000
75%,0.000000,1.000000,1.000000,55.000000,1.000000,1.000000,89.850000,1.000000
max,1.000000,1.000000,1.000000,72.000000,1.000000,1.000000,118.750000,1.000000


In [38]:
df.describe(exclude='number')

,gender,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaymentMethod,TotalCharges
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,2,3,3,3,3,3,3,3,3,3,4,6531
top,Male,0,Fiber optic,0,0,0,0,0,0,Month-to-month,Electronic check,20.2
freq,3555,3390,3096,3498,3088,3095,3473,2810,2785,3875,2365,11


In [5]:
X = df.drop(columns='Churn')
y = df['Churn'].replace({'Yes': 1, 'No':0}).to_numpy()
y = y.reshape(-1,1)

categories = X.describe(exclude='number').columns

enc = OrdinalEncoder()
scaler = MinMaxScaler() # Scaler added 6%
X = enc.fit_transform(X[categories])
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((5282, 16), (1761, 16), (5282, 1), (1761, 1))

In [42]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    
    model.add(Dense(50, input_dim=16, activation='relu'))
    
    model.add(Dense(20, activation='relu'))
    
    model.add(Dense(30, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
p = {'batch_size': [10, 20, 30, 50, 200], 'epochs': [20, 40, 60, 80], }
              
              
# # Create Grid Search
# grid = GridSearchCV(estimator=model, param_grid=p, n_jobs=1)
# grid_result = grid.fit(X_train, y_train)

# # Report Results
# print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print(f"Means: {mean}, Stdev: {stdev} with: {param}") 

In [43]:
model = KerasClassifier(build_fn=create_model, verbose=1, batch_size=50, epochs=60)
model.fit(X_train, y_train)

Train on 5282 samples
Epoch 1/60
5282/5282 [==============================] - 1s 176us/sample - loss: 0.4997 - accuracy: 0.7454
Epoch 2/60
5282/5282 [==============================] - 0s 88us/sample - loss: 0.4557 - accuracy: 0.7760
Epoch 3/60
5282/5282 [==============================] - 0s 71us/sample - loss: 0.4506 - accuracy: 0.7798
Epoch 4/60
5282/5282 [==============================] - 0s 61us/sample - loss: 0.4465 - accuracy: 0.7825
Epoch 5/60
5282/5282 [==============================] - 0s 91us/sample - loss: 0.4414 - accuracy: 0.7851
Epoch 6/60
5282/5282 [==============================] - 0s 76us/sample - loss: 0.4407 - accuracy: 0.7849
Epoch 7/60
5282/5282 [==============================] - 0s 58us/sample - loss: 0.4375 - accuracy: 0.7912
Epoch 8/60
5282/5282 [==============================] - 0s 75us/sample - loss: 0.4344 - accuracy: 0.7855
Epoch 9/60
5282/5282 [==============================] - 0s 82us/sample - loss: 0.4331 - accuracy: 0.7906
Epoch 10/60
5282/5282 [=========

In [44]:
y_pred = model.predict(X_test)
rounded = [round(x[0]) for x in y_pred]
y_pred1 = np.array(rounded, dtype='int64')
print(precision_score(y_test, y_pred1))
accuracy_score(y_test, y_pred1)

1761/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

0.7586598523566156

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?